In [7]:
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 자동으로 루트에서 찾아서 로드
api_key = os.getenv("OPENAI_API_KEY")

In [11]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings

# PDF 로딩
loader = PyPDFLoader("../data/sleep_performance.pdf")
documents = loader.load_and_split()

# 임베딩 생성
embedding = OpenAIEmbeddings()  # .env에 API 키 필요!
vectorstore = FAISS.from_documents(documents, embedding)

# 저장
vectorstore.save_local("vectorstores/sleep_rag")


In [13]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 벡터 DB 로딩
vectorstore = FAISS.load_local(
    "vectorstores/sleep_rag",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# LLM 설정
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# RAG 체인 생성
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 질의 실행
query = "문서의 내용을 요약해줘"
response = rag_chain(query)
print(response["result"])


이 연구는 수면부족이 운동 성능에 미치는 영향을 조사한 것으로, 670명의 참가자를 대상으로 45개의 연구를 분석하였습니다. 연구는 수면부족의 유형과 테스트 시간에 따라 성과에 미치는 영향을 살펴보았습니다. 수면부족은 최대 근력과 속도 성능에 영향을 미칠 수 있음을 보여주었으며, 이를 통해 수면 부족이 운동 능력에 미치는 다양한 측면을 이해하는 데 도움이 되었습니다. 연구는 표준화된 추출 형식을 사용하여 데이터를 관리하고, 방법론적 품질을 평가하기 위해 Cochrane Collaboration risk of bias (RoB) 2.0 도구를 사용했습니다. 결과는 수면부족이 운동 능력에 영향을 미칠 수 있음을 시사하며, 이를 통해 운동 선수와 건강한 비운동자 간의 차이, 수면부족 유형 및 테스트 시간에 따른 영향을 살펴보았습니다.
